In [205]:
import pandas as pd
import numpy as np

# Read the data
shows = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/complete_manually_cleaned_data.csv')
actor_rankings = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/actor_rankings.csv')
writer_rankings = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/writer_rankings.csv')
director_rankings = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/director_rankings.csv')

In [152]:
principals = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.principals.tsv', sep='\t')

In [177]:
crew = pd.read_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/title.crew.tsv', sep='\t')

In [178]:
# delete prinicpals with cateegoy not actor, director, or writer
principals2 = principals[principals['category'].isin(['actor','actress', 'self'])]

KeyboardInterrupt: 

In [206]:
# isolate tconst column of shows and merge with principals
shows_tconst = pd.DataFrame(shows['tconst'])
shows_tconst_director = shows_tconst.merge(crew, how='left', left_on='tconst', right_on='tconst')
shows_tconst_writer = shows_tconst.merge(crew, how='left', left_on='tconst', right_on='tconst')
shows_tconst_actor = shows_tconst.merge(principals2[principals2['category'].isin(['actor', 'actress', 'self'])][['tconst', 'nconst']], on='tconst', how='left')

In [207]:
shows_tconst_director['directors'] = shows_tconst_director['directors'].str.split(',')
shows_tconst_director = shows_tconst_director[['tconst', 'directors']].explode('directors')

In [208]:
shows_tconst_writer['writers'] = shows_tconst_writer['writers'].str.split(',')
shows_tconst_writer = shows_tconst_writer[['tconst', 'writers']].explode('writers')

In [209]:
# add rankings to each tconst/nconst pair
shows_tconst_director = shows_tconst_director.merge(director_rankings, left_on='directors', right_on='director', how='left')
shows_tconst_writer = shows_tconst_writer.merge(writer_rankings, left_on='writers', right_on='writer', how='left')
shows_tconst_actor = shows_tconst_actor.merge(actor_rankings, left_on='nconst', right_on='actor', how='left')

In [210]:
# take only top 3 actors per show
shows_tconst_actor = shows_tconst_actor.sort_values(by=['tconst', 'rank'], ascending=True).groupby('tconst').head(3)
# take only top 2 writers per show
shows_tconst_writer = shows_tconst_writer.sort_values(by=['tconst', 'rank'], ascending=True).groupby('tconst').head(3)
# take only top 1 directors per show
shows_tconst_director = shows_tconst_director.sort_values(by=['tconst', 'rank'], ascending=True).groupby('tconst').head(1)

In [211]:
# remove all columnds but tconst and rank
shows_tconst_director = shows_tconst_director[['tconst', 'rank']]
shows_tconst_writer = shows_tconst_writer[['tconst', 'rank']]
shows_tconst_actor = shows_tconst_actor[['tconst', 'rank']]

In [212]:
# group by tconst and average rank of all directors/writers/actors
shows_tconst_director = shows_tconst_director.groupby('tconst').mean().reset_index()
shows_tconst_writer = shows_tconst_writer.groupby('tconst').mean().reset_index()
shows_tconst_actor = shows_tconst_actor.groupby('tconst').mean().reset_index()

In [213]:
# merge with shows
shows = shows.merge(shows_tconst_director, on='tconst', how='left')
# rename rank column
shows = shows.rename(columns={'rank': 'director_rank'})
shows = shows.merge(shows_tconst_writer, on='tconst', how='left')
# rename rank column
shows = shows.rename(columns={'rank': 'writer_rank'})
shows = shows.merge(shows_tconst_actor, on='tconst', how='left')
# rename rank column
shows = shows.rename(columns={'rank': 'actor_rank'})

In [198]:
# sort shows by actor_rank
shows = shows.sort_values(by='actor_rank', ascending=True)

In [199]:
shows

,combined_title,release_year,release_date,is_top10,title,season,is_tv_show,tconst,averageRating,numVotes,titleType,runtimeMinutes,genres,director_rank,writer_rank,actor_rank
509,Don't Look Up,2021,20211224,True,dont look up,NaN,False,tt11286314,7.2,526805.0,movie,138,"Comedy,Drama,Sci-Fi",253.0,110.000000,35.333333
1831,The Last Thing He Wanted,2020,20200221,True,the last thing he wanted,NaN,False,tt7456312,4.3,15507.0,movie,115,"Crime,Drama,Thriller",2209.0,12727.666667,36.333333
1582,Space Force: Season 2,2022,20220218,True,space force,2.0,True,tt9612516,6.7,64650.0,tvSeries,30,Comedy,361.0,317.000000,94.666667
1581,Space Force: Season 1,2020,20200529,True,space force,1.0,True,tt9612516,6.7,64650.0,tvSeries,30,Comedy,361.0,317.000000,94.666667
1678,The Adam Project,2022,20220311,True,the adam project,NaN,False,tt2463208,6.7,194289.0,movie,106,"Action,Adventure,Comedy",133.0,2335.333333,116.666667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2127,WWII in Color: Road to Victory: Season 1,2021,20211222,True,wwii in color road to victory,1.0,True,tt16477402,8.2,1845.0,tvSeries,50,"Documentary,History,War",65376.5,4.000000,NaN
2137,We Are: The Brooklyn Saints: Season 1,2021,20210129,False,we are the brooklyn saints,1.0,True,tt13656220,6.5,164.0,tvSeries,\N,"Documentary,Sport",16435.5,4.000000,NaN
2154,White Hot: The Rise & Fall of Abercrombie & Fitch,2022,20220419,True,white hot the rise fall of abercrombie fitch,NaN,False,tt19034522,5.7,4483.0,movie,88,Documentary,12744.0,4.000000,NaN
2164,Why Did You Kill Me?,2021,20210414,False,why did you kill me,NaN,False,tt14318430,5.6,4292.0,movie,83,"Crime,Documentary",NaN,4.000000,NaN


In [216]:
# isolate movies    
movies = shows[shows['titleType'] == 'movie']

In [217]:
movies

,combined_title,release_year,release_date,is_top10,title,season,is_tv_show,tconst,averageRating,numVotes,titleType,runtimeMinutes,genres,director_rank,writer_rank,actor_rank
2,10 Days With Santa Claus,2020,20201204,False,10 days with santa claus,NaN,False,tt10803866,6.1,856.0,movie,100,"Comedy,Family",16005.5,32863.500000,14106.166667
4,1000 Miles from Christmas,2021,20211224,True,1000 miles from christmas,NaN,False,tt13782052,5.8,1531.0,movie,102,"Comedy,Romance",13557.5,29730.500000,31277.833333
5,11M: Terror in Madrid,2022,20220223,True,11m terror in madrid,NaN,False,tt6275154,6.9,1928.0,movie,92,Documentary,50497.0,NaN,NaN
7,137 Shots,2021,20211215,True,137 shots,NaN,False,tt16379224,6.3,932.0,movie,104,"Crime,Documentary",101870.5,NaN,487505.000000
8,13: The Musical,2022,20220812,False,13 the musical,NaN,False,tt3975918,5.2,621.0,movie,91,"Comedy,Drama,Family",567.0,30261.500000,6840.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2213,Your Name Engraved Herein,2020,20201223,False,your name engraved herein,NaN,False,tt10329134,7.2,6215.0,movie,118,"Drama,Romance",17164.5,35384.000000,27692.000000
2218,Zero to Hero,2021,20211105,True,zero to hero,NaN,False,tt14892216,6.7,925.0,movie,102,"Biography,Drama,Sport",55914.5,NaN,19532.166667
2220,"ariana grande: excuse me, i love you",2020,20201221,False,ariana grande excuse me i love you,NaN,False,tt13615386,6.3,2648.0,movie,97,"Documentary,Music",5695.0,NaN,47846.666667
2223,"tick, tick...BOOM!",2021,20211119,True,tick tickboom,NaN,False,tt8721424,7.5,103877.0,movie,120,"Biography,Drama,Musical",129755.5,10385.000000,2361.333333


In [218]:
print('director_count: ', movies['director_rank'].isna().sum())
print('writer_count: ', movies['writer_rank'].isna().sum())
print('actor_count: ', movies['actor_rank'].isna().sum())
# total number of movies
print('total: ', len(movies))

director_count:  47
writer_count:  111
actor_count:  28
total:  741


director_count:  89
writer_count:  70
actor_count:  102
total:  2226

In [214]:
# write shows to csv
shows.to_csv('/Users/zachwayne/Dropbox (MIT)/Netflix - Seize the Data/showsWithPrinicpalRankings.csv', index=False)